**import main libs**

In [1]:
import os
import faker
import random
import numpy as np
import pandas as pd

***creating the test sales data structure***

In [2]:
sales_data = {"year": [], "month": [], "sales": [], "expenses": []}

***using faker to create the sales data set***

In [3]:
fake_data = faker.Faker()
for _ in range(500): # increase the range to generate more records
    sales_data["year"].append(random.randint(1999, 2022)) # or simply use range(6, 106)
    sales_data["month"].append(random.choice(['Jan', 'Mar', 'Dec']))
    sales_data["sales"].append(fake_data.random_int())
    sales_data["expenses"].append(fake_data.random_int())

***creating the pandas data frame***

In [4]:
sales_data = pd.DataFrame(sales_data)
sales_data.head()

,year,month,sales,expenses
0,2003,Dec,7115,452
1,2000,Jan,8848,6888
2,2013,Dec,2712,5924
3,2001,Jan,50,7669
4,2019,Dec,5061,6291


In [5]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   year      500 non-null    int64 
 1   month     500 non-null    object
 2   sales     500 non-null    int64 
 3   expenses  500 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 15.8+ KB


***Data Transformation - new balance column from diff of sales & expenses***

In [6]:
sales_data['balance'] = sales_data.apply(lambda row: row.sales - row.expenses, axis=1)
sales_data.head()

,year,month,sales,expenses,balance
0,2003,Dec,7115,452,6663
1,2000,Jan,8848,6888,1960
2,2013,Dec,2712,5924,-3212
3,2001,Jan,50,7669,-7619
4,2019,Dec,5061,6291,-1230


***Data Transformation - new is_profit column looking at balance > 0***

In [7]:
sales_data['is_profit'] = sales_data['balance'].map(lambda balance: True if balance > 0 else False)
sales_data.iat[498, 4] =  202
sales_data[sales_data.balance == 202]

,year,month,sales,expenses,balance,is_profit
498,2007,Mar,5212,141,202,True


In [8]:
sales_data.head()

,year,month,sales,expenses,balance,is_profit
0,2003,Dec,7115,452,6663,True
1,2000,Jan,8848,6888,1960,True
2,2013,Dec,2712,5924,-3212,False
3,2001,Jan,50,7669,-7619,False
4,2019,Dec,5061,6291,-1230,False


***Perform pivot_table on sales data for summing up year wise***

In [9]:
sales_data = pd.pivot_table(sales_data, values=['sales', 'expenses', 'balance'], index='year', fill_value=0, aggfunc='sum')
sales_data.head()

,balance,expenses,sales
year,,,
1999,-25186,124434,99248
2000,20954,81953,102907
2001,-45595,114751,69156
2002,35680,84950,120630
2003,-19129,75607,56478


In [10]:
sales_data['result'] = sales_data.apply(lambda row: 'Loss' if row.balance < 0 else 'Profit', axis=1)
sales_data.head()

,balance,expenses,sales,result
year,,,,
1999,-25186,124434,99248,Loss
2000,20954,81953,102907,Profit
2001,-45595,114751,69156,Loss
2002,35680,84950,120630,Profit
2003,-19129,75607,56478,Loss


***Perform filtering on sales data: year 2005***

In [11]:
sales_data[sales_data.index == 2005]

,balance,expenses,sales,result
year,,,,
2005,-5915,142802,136887,Loss


***Perform filtering on sales data: balance >= 15000***

In [12]:
sales_data[sales_data.balance >= 15000]

,balance,expenses,sales,result
year,,,,
2000,20954,81953,102907,Profit
2002,35680,84950,120630,Profit
2009,17811,93948,111759,Profit


In [16]:
sales_data.loc[[2020,2022]]

,balance,expenses,sales,result
year,,,,
2020,-3697,129635,125938,Loss
2022,-326,124070,123744,Loss


In [19]:
sales_data.loc[sales_data.balance > 500]

,balance,expenses,sales,result
year,,,,
2000,20954,81953,102907,Profit
2002,35680,84950,120630,Profit
2007,14022,73575,92466,Profit
2009,17811,93948,111759,Profit
2011,11976,103692,115668,Profit
2017,13825,104011,117836,Profit
2018,601,108684,109285,Profit
2019,14097,104788,118885,Profit
2021,2188,146809,148997,Profit


In [15]:
sales_group = sales_data.groupby('result')
for group, group_df in sales_group:
    print(f"\nPerform grouping on sales data: {group}")
    group_df['severity'] = pd.cut(group_df['balance'], bins=[-75000, -50000, -25000, 0, 25000, 50000, 75000, 95000], labels=['Worst', 'Bad', 'Sick', 'Ok', 'Well', 'Good', 'Best'])
    print(group_df)


Perform grouping on sales data: Loss
      balance  expenses   sales result severity
year                                           
1999   -25186    124434   99248   Loss      Bad
2001   -45595    114751   69156   Loss      Bad
2003   -19129     75607   56478   Loss     Sick
2004   -24125    100409   76284   Loss     Sick
2005    -5915    142802  136887   Loss     Sick
2006    -9685    117476  107791   Loss     Sick
2008   -21294     93149   71855   Loss     Sick
2010   -50179    163310  113131   Loss    Worst
2012    -7697     76194   68497   Loss     Sick
2013   -23079    112407   89328   Loss     Sick
2014      -65     97890   97825   Loss     Sick
2015    -5787     98862   93075   Loss     Sick
2016   -13190     94028   80838   Loss     Sick
2020    -3697    129635  125938   Loss     Sick
2022     -326    124070  123744   Loss     Sick

Perform grouping on sales data: Profit
      balance  expenses   sales  result severity
year                                            
2000    

***Data Transformation - sales data:to binary***

In [14]:
sales_data.loc[sales_data.balance > 0, "balance"] = 1
sales_data.loc[sales_data.balance <= 0, "balance"] = 0
sales_data.loc[sales_data.balance > 0, ["sales", "expenses"]] = 1
sales_data.loc[sales_data.balance <= 0, ["sales", "expenses"]] = 0
sales_data

,balance,expenses,sales,result
year,,,,
1999,0,0,0,Loss
2000,0,0,0,Loss
2001,0,0,0,Loss
2002,0,0,0,Loss
2003,1,1,1,Profit
2004,0,0,0,Loss
2005,1,1,1,Profit
2006,1,1,1,Profit
2007,1,1,1,Profit


In [24]:
ser = pd.Series(np.random.rand(5))
print(type(ser))
ser

<class 'pandas.core.series.Series'>


0    0.686227
1    0.602473
2    0.816133
3    0.176602
4    0.126674
dtype: float64

In [33]:
df = pd.DataFrame(np.random.rand(50,5), index=np.arange(50), columns=list("abcde"))
df.head()

,a,b,c,d,e
0,0.898900,0.284445,0.975144,0.516018,0.207191
1,0.285208,0.517077,0.301079,0.185226,0.218211
2,0.204987,0.269925,0.017050,0.404500,0.711959
3,0.100358,0.462952,0.455381,0.990401,0.188913
4,0.765669,0.561169,0.835971,0.779373,0.906189
